In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import sys

if not ".." in sys.path:
    sys.path.append("..")

In [3]:
from tqdm import tqdm

import torch
from lit_modules.litmodels import LitMemNet, LitResNet50
from lit_modules.litdatamodules.lit_lamem import LitLaMemDataModule

In [4]:
RESNET_CHECKPOINT = "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/resnet_lalem/lightning_logs/version_0/checkpoints/epoch=499-step=88000.ckpt"
MEMNET_CHECKPOINT = (
    "https://github.com/andrewrkeyes/Memnet-Pytorch-Model/raw/master/model.ckpt"
)

In [5]:
checkpoint = torch.utils.model_zoo.load_url(MEMNET_CHECKPOINT)

In [6]:
memnet = LitMemNet(checkpoint=checkpoint)
resnet = LitResNet50.load_from_checkpoint(RESNET_CHECKPOINT)

memnet.eval()
resnet.eval()

print("Completed")

/lustre06/project/6067616/soroush1/pretrain-imagenet/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Completed


In [7]:
root = "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/data/LaMem/lamem_images/lamem"
dataset = LitLaMemDataModule(root=root, batch_size=128, num_workers=3)
dataset.setup("TEST")

test_loader = dataset.test_dataloader()

In [ ]:
criterion = torch.nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
memnet.to(device)
resnet.to(device)

losses = []
for i, (x, y) in tqdm(enumerate(test_loader)):
    x = x.to(device)
    y = y.to(device)

    memnet_output = memnet(x)
    memnet_output = torch.nn.functional.sigmoid(memnet_output)

    resnet_output = resnet(x)

    loss = criterion(resnet_output.squeeze(), memnet_output.squeeze())
    losses.append(loss)

print("Completed")

1it [00:18, 18.72s/it]